In [1]:
import psycopg2 as pg
import ipywidgets as widgets

In [2]:
conn = pg.connect(dbname="sec", user="secapp", host="r_363-postgres-1")
curr = conn.cursor()

get the undifferentiated terms

In [ ]:
substage = widgets.Dropdown(
    options=[("IVA", "\yiva\y"), ("IVB", "\yivb\y"), ("IVC", "\yivc\y")],
    description="Substage",
)
substage

In [ ]:
substage_regexp = substage.value
curr.execute(
    "select pref_name,code from hnc_stages where pref_name ~* %s and pref_name ~* 'undiff'",
    (substage_regexp,),
)
undiff_substage_terms = curr.fetchall()
for row in undiff_substage_terms:
    print(row)

In [ ]:
undiff_code = widgets.Dropdown(
    options=undiff_substage_terms,
    description='Substage',
)
undiff_code

locate the higher order stage terms for the undifferentiated terms

In [ ]:
selected_name = [t[0] for t in undiff_substage_terms if t[1] == undiff_code.value][0]
keyword = widgets.Dropdown(
    options=selected_name.split(),
    description='Keyword',
)
keyword

check if any of the higher order stage terms are parents of the undiff terms

In [ ]:
selected_substage = [t[0] for t in undiff_substage_terms if t[1] == undiff_code.value][
    0
]
print("Selected undiff term ----")
print(undiff_code.value, selected_substage, '\n')

curr.execute(
    "select code, pref_name from hnc_stages where pref_name ~* '\yiv\y' and pref_name ~* %s",
    (keyword.value,),
)
higher_order_stage_terms = curr.fetchall()
print(f"Possible {keyword.value} parents ----")
for row in higher_order_stage_terms:
    print(row)

print("\nFound paths ----")
for row in higher_order_stage_terms:
    parent = row[0]
    curr.execute(
        """select path from ncit_tc_with_path where parent = %s and descendant = %s""",
        (parent, undiff_code.value),
    )
    paths = curr.fetchall()
    for path in paths:
        codes = path[0].split("|")
        t = 0
        for code in codes:
            curr.execute("select code, pref_name from ncit where code = %s", (code,))
            term = curr.fetchone()
            print("\t" * t, term)
            t += 1

In [6]:
conn.rollback()

In [ ]:
curr.close()
conn.close()